In [1]:
# Import the modules
import datetime
import pathlib
import urllib
import os

import numpy as np
import spiceypy

In [2]:
# Load the SPICE kernels via a meta file
spiceypy.furnsh('kernel_meta.txt')

# Create an initial date-time object that is converted to a string
datetime_utc = datetime.datetime(year=2021, month=11, day=21).strftime('%Y-%m-%dT%H:%M:%S')

# Convert to Ephemeris Time (ET) using the SPICE function utc2et
datetime_et = spiceypy.utc2et(datetime_utc)

In [3]:
# Get the G*M value for the Sun
_, gm_sun_pre = spiceypy.bodvcd(bodyid=10, item='GM', maxn=1)

GM_SUN = gm_sun_pre[0]

In [4]:
# On spaceweather.com we can see that an asteroid has a close Earth fly-by:
# Orpheus on 2021-November-21.
#
# Will the encounter alter the orbit of the asteroid? Let's have a first look
# on the so-called sphere of influence (SOI) of our planet.
# A simple model assumes that the SOI is a sphere. The semi major axis is set
# to 1 AU:

# 1 AU in km
ONE_AU = spiceypy.convrt(x=1, inunit='AU', outunit='km')

# Set the G*M parameter of our planet
_, gm_earth_pre = spiceypy.bodvcd(bodyid=399, item='GM', maxn=1)
GM_EARTH = gm_earth_pre[0]

# Compute the SOI radius of the Earth
SOI_EARTH_R = ONE_AU * (GM_EARTH/GM_SUN) ** (2.0/5.0)

# Set one Lunar Distance (LD) in km (value from spaceweather.com)
ONE_LD = 384401.0

print(f'SOI of the Earth in LD: {SOI_EARTH_R/ONE_LD}')

SOI of the Earth in LD: 2.4054224328225597


In [5]:
# Let's obtain the orbit elements data of 3361 Orpheus from 
# https://ssd.jpl.nasa.gov/tools/sbdb_lookup.html#/?sstr=3361&view=OPD

# Before we compute a state vector of the asteroid and the current distance
# to our home planet we need to define a function to round the data. A common
# convention for scientific work is to round the data to one significant
# digit. We create a lambda function that rounds the values based on the
# provided measurement error
round_sig = lambda value, err: np.round(value, -1*(int(np.floor(np.log10(err)))))

In [6]:
# Set now the perihelion in km
neo_orpheus_perihelion_km = spiceypy.convrt(round_sig(0.8193931144261904, \
                                                      4.396E-8), \
                                            inunit='AU', outunit='km')

# Set the eccentricity
neo_orpheus_ecc = round_sig(0.3231489803944947, 3.6326E-8)

# Set the inclination, longitude of ascending node and argument of periapsis
# in radians
neo_orpheus_inc_rad = np.radians(round_sig(2.661237238614012, 3.5526E-6))
neo_orpheus_lnode_rad = np.radians(round_sig(188.6885422918818, 3.8154E-5))
neo_orpheus_argp_rad = np.radians(round_sig(302.3633807683478, 3.7866E-5))

# Set the mean anomaly and corresponding epoch in Julian Date (JD)
neo_orpheus_m0_at_t0_rad = np.radians(round_sig(4.38004009432731, 5.0726E-6))
neo_orpheus_t0 = spiceypy.utc2et('2459600.5 JD')

In [7]:
# Set the orbital elements array
neo_orpheus_orbital_elements = [neo_orpheus_perihelion_km, \
                                neo_orpheus_ecc, \
                                neo_orpheus_inc_rad, \
                                neo_orpheus_lnode_rad, \
                                neo_orpheus_argp_rad, \
                                neo_orpheus_m0_at_t0_rad, \
                                neo_orpheus_t0, \
                                GM_SUN]

# Compute the state vector
neo_orpheus_state_vector = spiceypy.conics(neo_orpheus_orbital_elements, datetime_et)

print(f'Current state vector of Orpheus in km and km/s ({datetime_utc})):\n' \
      f'{neo_orpheus_state_vector}')

Current state vector of Orpheus in km and km/s (2021-11-21T00:00:00)):
[ 7.94813557e+07  1.25316332e+08 -5.19987510e+06 -3.11328503e+01
  9.22252838e+00 -6.42351184e-01]


In [8]:
# Now compute the state vector of the Earth:
earth_state_vector, _ = spiceypy.spkgeo(targ=399, \
                                        et=datetime_et, \
                                        ref='ECLIPJ2000',
                                        obs=10)

# Compute the current distance of the Earth and the asteroids in LD
earth_orpheus_dist_km = spiceypy.vnorm(earth_state_vector[:3] \
                                      - neo_orpheus_state_vector[:3])
print(f'Current distance between the Earth and Orpheus ({datetime_utc}):\n' \
      f'{earth_orpheus_dist_km / ONE_LD} LD')

Current distance between the Earth and Orpheus (2021-11-21T00:00:00):
15.09005615222323 LD
